# FEATURE SELECTION
### 1. RECURSIVE FEATURE SELECTION
### 2. FEATURE IMPORTANCE (TREE BASED)
### 3. SELECT KBEST ( UNIVARIATE)
### 4. SELECT FROM MODEL (L1)
### 5. LOW VARIANCE (BASED ON SCORE)

In [58]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.feature_selection import RFE, f_regression
from sklearn.linear_model import (LinearRegression, Ridge, Lasso, RandomizedLasso)
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.feature_selection import SelectFromModel
import sklearn 
from sklearn import preprocessing 
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import VarianceThreshold
from matplotlib import pyplot
#from xgboost import plot_importance
#from sklearn.feature_selection import chi2
#from sklearn.feature_selection import f_classif
#from sklearn.feature_selection import f_regression

In [59]:
url = "https://raw.githubusercontent.com/LuisM78/Appliances-energy-prediction-data/master/energydata_complete.csv"
df = pd.read_csv(url)
df['date'] = pd.to_datetime(df['date'])
df['month'] , df['time'] , df['hour'] ,df['day'] , df['day_of_week'],df['Numerical_Week'] =  df['date'].dt.month , df['date'].dt.time , df['date'].dt.hour , df['date'].dt.day , df['date'].dt.weekday_name,df['date'].dt.weekday
df['weekStatus'] = df['date'].dt.dayofweek
df['WeekStatus'] = np.where(df['weekStatus'] < 5, 'Weekday', 'Weekend')
#df['DateStr'] = df['date'].apply(lambda x: x.strftime('%Y%m%d%H%M'))
#df.head(5)

# converting the time to seconds , appending that column to the existing dataframe and then making the 'date' column as index to 
# the dataset.

d = df.date[0:len(df.date)]
data = []
#print(data)
for i in range (len(d)):
    if (i==0):
        a = 61200
        data.append(a)
    elif (i > 0 ):
        a = a + 600
        data.append(a)
#print((data))
#df['data_converted'] = data
df1 = pd.DataFrame({'data_converted' : data})
df['NSM'] = df1

#df['data_converted'] = df.astype(int)
#df.columns
df = df.set_index('date')


In [60]:

#df = df.drop(['date'] ,axis = 1)
df=df.drop(['day_of_week'] ,axis = 1)
df=df.drop(['WeekStatus'] ,axis = 1)
df=df.drop(['time'] ,axis = 1)

#df=df.drop(['data_converted'] ,axis = 1)

#df = df.astype('float32')
#scaler = MinMaxScaler(feature_range=(0, 1))
#df = scaler.fit_transform(df)

In [61]:
Y = df.Appliances.values


df = df.drop(['Appliances'] , axis = 1)

X = df.as_matrix()
features = df.columns

### 1. RECURSIVE FEATURE ELIMINATION :

In [47]:
estimator = LinearRegression()
selector = RFE(estimator , 22 , step=1 )
selector = selector.fit(X,Y)
#selector.support_
ranking = selector.ranking_
#coeff = selector.coef_
#fi = selector.feature_importances_
print (sorted(zip(map(float, ranking), features)))
#print(sorted(zip(map(float, coeff), features)))
#print("           ")
#print(sorted(zip(map(float, fi), features)))
#print('ranking')
prediction = selector.predict(X)
scores = r2_score(Y , prediction)
print("            ")
print(scores)


[(1.0, 'Numerical_Week'), (1.0, 'RH_1'), (1.0, 'RH_2'), (1.0, 'RH_3'), (1.0, 'RH_4'), (1.0, 'RH_7'), (1.0, 'RH_8'), (1.0, 'T1'), (1.0, 'T2'), (1.0, 'T3'), (1.0, 'T4'), (1.0, 'T6'), (1.0, 'T7'), (1.0, 'T8'), (1.0, 'T9'), (1.0, 'T_out'), (1.0, 'Tdewpoint'), (1.0, 'Windspeed'), (1.0, 'hour'), (1.0, 'lights'), (1.0, 'month'), (1.0, 'weekStatus'), (2.0, 'T5'), (3.0, 'RH_9'), (4.0, 'RH_out'), (5.0, 'Visibility'), (6.0, 'day'), (7.0, 'Press_mm_hg'), (8.0, 'RH_6'), (9.0, 'rv1'), (10.0, 'rv2'), (11.0, 'RH_5'), (12.0, 'NSM')]
            
0.169190645744


In [63]:
estimator = RandomForestRegressor()
selector = RFE(estimator ,20, step=1 )
selector = selector.fit(X,Y)
#selector.support_
ranking = selector.ranking_
print (sorted(zip(map(float, ranking), features)))
print('            ')
prediction = selector.predict(X)
scores = r2_score(Y , prediction)
print(scores)

[(1.0, 'NSM'), (1.0, 'Press_mm_hg'), (1.0, 'RH_1'), (1.0, 'RH_2'), (1.0, 'RH_3'), (1.0, 'RH_4'), (1.0, 'RH_5'), (1.0, 'RH_6'), (1.0, 'RH_7'), (1.0, 'RH_8'), (1.0, 'RH_out'), (1.0, 'T2'), (1.0, 'T3'), (1.0, 'T5'), (1.0, 'T8'), (1.0, 'T_out'), (1.0, 'Tdewpoint'), (1.0, 'hour'), (1.0, 'lights'), (1.0, 'rv2'), (2.0, 'RH_9'), (3.0, 'Windspeed'), (4.0, 'T7'), (5.0, 'T4'), (6.0, 'T6'), (7.0, 'Visibility'), (8.0, 'T1'), (9.0, 'rv1'), (10.0, 'day'), (11.0, 'T9'), (12.0, 'weekStatus'), (13.0, 'Numerical_Week'), (14.0, 'month')]
            
0.918020954333


### 2. FEATURE IMPORTANCE(TREE BASED) - RANDOM FOREST

In [48]:
trees = ExtraTreesRegressor(n_estimators=250,
                              random_state=0)

trees = trees.fit(X, Y)
importances = trees.feature_importances_
print(sorted(zip(map(float ,importances),features)))
print('            ')
prediction = trees.predict(X)
#scores = oob_score(Y ,prediction)
#print(scores)

[(0.016246955592418733, 'month'), (0.018709158983822062, 'T9'), (0.0193705098715039, 'Numerical_Week'), (0.019643934750393395, 'weekStatus'), (0.021117532265162893, 'day'), (0.0219977219606682, 'rv2'), (0.02228638689572151, 'T7'), (0.022349244119737842, 'rv1'), (0.023915764574716714, 'Visibility'), (0.024266931987183212, 'T5'), (0.02448226921996136, 'NSM'), (0.024773608894046565, 'RH_7'), (0.02533353404456651, 'Tdewpoint'), (0.02550190523041148, 'T_out'), (0.02558887317395682, 'RH_4'), (0.025594506469101864, 'Press_mm_hg'), (0.025913936815239253, 'T4'), (0.02606494850108655, 'RH_6'), (0.02670825251714159, 'RH_9'), (0.026874839200705807, 'T1'), (0.027411203014314364, 'RH_8'), (0.027567196824910916, 'T2'), (0.028712534411888056, 'T6'), (0.029538612666353773, 'RH_5'), (0.029806073535463943, 'T8'), (0.03122967901481279, 'RH_1'), (0.031706556443908705, 'Windspeed'), (0.032881951008522654, 'RH_2'), (0.03353832680553693, 'RH_3'), (0.034845374865078445, 'RH_out'), (0.037251048968137954, 'light

### 3. SELECT KBEST(UNIVARIATE)

In [67]:
best_variables = SelectKBest(f_regression,k=20)                        #f_classif , chi2 , f_regression
best_variables.fit(X,Y)
best_variables = best_variables.fit_transform(X,Y)
#print(zip(map(float ,best_variables) ,features))
best_variables
#best_variables[:1]
#print(i for i in (zip(map(float ,best_variables) ,features)))

'''
f_classif
ANOVA F-value between label/feature for classification tasks.

mutual_info_classif
Mutual information for a discrete target.

chi2
Chi-squared stats of non-negative features for classification tasks.

f_regression
F-value between label/feature for regression tasks.

mutual_info_regression
Mutual information for a continuous target.

SelectPercentile
Select features based on percentile of the highest scores.

SelectFpr
Select features based on a false positive rate test.

SelectFdr
Select features based on an estimated false discovery rate.

SelectFwe
Select features based on family-wise error rate.

GenericUnivariateSelect
Univariate feature selector with configurable mode.

'''

array([[ 30.        ,  19.89      ,  47.59666667, ...,  92.        ,
          7.        ,  17.        ],
       [ 30.        ,  19.89      ,  46.69333333, ...,  92.        ,
          6.66666667,  17.        ],
       [ 30.        ,  19.89      ,  46.3       , ...,  92.        ,
          6.33333333,  17.        ],
       ..., 
       [ 10.        ,  25.5       ,  46.59666667, ...,  56.33333333,
          3.66666667,  17.        ],
       [ 10.        ,  25.5       ,  46.99      , ...,  56.66666667,
          3.83333333,  17.        ],
       [ 10.        ,  25.5       ,  46.6       , ...,  57.        ,
          4.        ,  18.        ]])

### 4. SELECT FROM MODEL(L1) - LINEAR REGRESSION

In [62]:
lasso = Lasso(alpha=.05)
lasso.fit(X, Y)
Lasso = (np.abs(lasso.coef_), features)
print (Lasso)

(array([  1.92202561e+00,   3.75236632e+00,   1.45430896e+01,
         1.80860820e+01,   1.31094332e+01,   2.71830133e+01,
         4.12038495e+00,   2.20257987e+00,   1.01738977e+00,
         4.34037649e-02,   4.29283515e-02,   6.86899286e+00,
         4.34249006e-02,   6.29268266e-01,   1.50454528e+00,
         7.08519289e+00,   3.73542439e+00,   9.06159302e+00,
         6.37288879e-01,   7.31765826e+00,   1.42342007e-01,
         2.43972724e-01,   1.48410757e+00,   1.55961968e-01,
         2.40097934e+00,   4.27670800e-02,   2.10533855e-17,
         8.86260781e+00,   9.84385321e-01,   5.58805364e-01,
         1.59417650e+00,   2.04845823e-13,   8.27707964e-06]), Index(['lights', 'T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3', 'T4', 'RH_4', 'T5',
       'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8', 'RH_8', 'T9', 'RH_9', 'T_out',
       'Press_mm_hg', 'RH_out', 'Windspeed', 'Visibility', 'Tdewpoint', 'rv1',
       'rv2', 'month', 'hour', 'day', 'Numerical_Week', 'weekStatus', 'NSM'],
      dtyp

C:\Users\Dell-pc\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


### 5. LOW VARIANCE (BASED ON SCORE)
#### not preferred beacuse it considers only the X variables and not Y. So genearlly used for unsupervised methods.

In [43]:
desired_features = VarianceThreshold(threshold=(.2 * (1 - 0.0)))
desired_features.fit_transform(X)


array([[  3.00000000e+01,   1.98900000e+01,   4.75966667e+01, ...,
          0.00000000e+00,   0.00000000e+00,   6.12000000e+04],
       [  3.00000000e+01,   1.98900000e+01,   4.66933333e+01, ...,
          0.00000000e+00,   0.00000000e+00,   6.18000000e+04],
       [  3.00000000e+01,   1.98900000e+01,   4.63000000e+01, ...,
          0.00000000e+00,   0.00000000e+00,   6.24000000e+04],
       ..., 
       [  1.00000000e+01,   2.55000000e+01,   4.65966667e+01, ...,
          4.00000000e+00,   4.00000000e+00,   1.19004000e+07],
       [  1.00000000e+01,   2.55000000e+01,   4.69900000e+01, ...,
          4.00000000e+00,   4.00000000e+00,   1.19010000e+07],
       [  1.00000000e+01,   2.55000000e+01,   4.66000000e+01, ...,
          4.00000000e+00,   4.00000000e+00,   1.19016000e+07]])

### 1. Out of these methods only Recursive Feature Elimination ( Random Forest Regressor) has proved to be best. 
### 2. Others such SelectKBest , Based on variance are univariate models. 
### 3. L1 based feature selection doesn't provide good output.
### 4. ExtraTreeRegressor is good , but the output when compared with other automated feature selection tools doesn't match.
### 5. Recursive Feature Elimination(Random Forest) is selected over Recursive Feature Elimination (Linear Regression) because the variance of the former is very high compared to latter.